In [1]:
import torch
from torch import nn
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("AppleHistoricalQuotes.csv", index_col="Date")

In [3]:
data.replace("[$]", "", regex=True, inplace=True)
data.head(10)

,Close/Last,Volume,Open,High,Low
Date,,,,,
02/28/2020,273.36,106721200,257.26,278.41,256.37
02/27/2020,273.52,80151380,281.1,286,272.96
02/26/2020,292.65,49678430,286.53,297.88,286.5
02/25/2020,288.08,57668360,300.95,302.53,286.13
02/24/2020,298.18,55548830,297.26,304.18,289.23
02/21/2020,313.05,32426420,318.62,320.45,310.5
02/20/2020,320.3,25141490,322.63,324.65,318.21
02/19/2020,323.62,23495990,320,324.57,320
02/18/2020,319,38190550,315.36,319.75,314.61


In [4]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

prices = data.iloc[:, 0].values
prices = np.array(prices, dtype=np.float)

scaler = MinMaxScaler(feature_range=(-1, 1))
prices = scaler.fit_transform(prices.reshape(-1, 1))

In [5]:
def split_data(stock, lookback):
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(stock) - lookback): 
        data.append(stock[index: index + lookback])
    
    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]

In [6]:
lookback = 20
data = []
for index in range(len(prices) - lookback): 
        data.append(prices[index: index + lookback])
data = np.array(data)
x = data[:,:-1,:]
y = data[:,-1,:]

In [7]:
x = torch.from_numpy(x).type(torch.Tensor)
y = torch.from_numpy(y).type(torch.Tensor)

In [8]:
input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1
company_name = "TSLA"

In [9]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn) = self.gru(x, (h0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [10]:
gru_model = GRU(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)

In [11]:
checkpoint = torch.load(f"checkpoints/{company_name}/gru32.pth")

In [12]:
gru_model.load_state_dict(checkpoint)

<All keys matched successfully>

In [13]:
import math, time
from sklearn.metrics import mean_squared_error

# make predictions
y_pred = gru_model(x)

# invert predictions
y = scaler.inverse_transform(y.detach().numpy())
y_pred = scaler.inverse_transform(y_pred.detach().numpy())

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y[:,0], y_pred[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))

Train Score: 5.95 RMSE


In [14]:
# shift train predictions for plotting
PredictPlot = np.empty_like(prices)
PredictPlot[:, :] = np.nan
PredictPlot[lookback:len(y_pred)+lookback, :] = y_pred

original = scaler.inverse_transform(prices.reshape(-1,1))

predictions = np.append(PredictPlot, original, axis=1)
result = pd.DataFrame(predictions)

In [15]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=result.index, y=result[1],
                    mode='lines',
                    name='Actual Value'))
fig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[0],
                    mode='lines',
                    name='Prediction')))
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        title_text='Close (USD)',
        titlefont=dict(
            family='Rockwell',
            size=12,
            color='black',
        ),
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='black',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Rockwell',
            size=12,
            color='black',
        ),
    ),
    showlegend=True,
    template = 'plotly_white'

)



annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text=f'Results (GRU32) of {company_name} on AAPL',
                              font=dict(family='Rockwell',
                                        size=26,
                                        color='black'),
                              showarrow=False))
fig.update_layout(annotations=annotations)

fig.show()